In [41]:
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [42]:
 # Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/daisyzhang/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache


## NASA Mars News

In [43]:
url = "https://redplanetscience.com/"
browser.visit(url)

In [44]:
html = browser.html
soup = bs(html, 'html.parser')

In [45]:
# scraping the news title with tage and class name 
titles = soup.find_all('div', class_='content_title')[0].text
titles

"HiRISE Views NASA's InSight and Curiosity on Mars"

In [46]:
# scraping news paragraph 
paragraphs  = soup.find_all('div', class_='article_teaser_body')[0].text
paragraphs

'New images taken from space offer the clearest orbital glimpse yet of InSight as well as a view of Curiosity rolling along.'

## JPL Mars Space Images - Featured Image

In [10]:
jpl_url = "https://spaceimages-mars.com/"
browser.visit(jpl_url)

In [11]:
html = browser.html
soup = bs(html, 'html.parser')

In [12]:
featured_image_url = soup.find('img',class_="headerimage fade-in")['src']

In [13]:
featured_image_url

'image/featured/mars1.jpg'

## Mars Facts

In [14]:
fact_url = "https://galaxyfacts-mars.com/"
response = requests.get(fact_url)
soup = bs(response.text, 'html.parser')

In [15]:
# read from url
tables = pd.read_html(fact_url)
tables

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [16]:
fact_df=tables[0]
fact_df.head()

fact_df = fact_df.rename(columns={0:"Mars - Earth Comparison",1:"Mars",2:"Earth"})

In [17]:
fact_df = fact_df.drop([0, 0])
fact_df.set_index("Mars - Earth Comparison",inplace=True)

In [18]:
fact_df

,Mars,Earth
Mars - Earth Comparison,,
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [19]:
# generate HTML tables from DataFrames
html_table = fact_df.to_html()
html_table.replace('\n', '')

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Mars</th>      <th>Earth</th>    </tr>    <tr>      <th>Mars - Earth Comparison</th>      <th></th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Diameter:</th>      <td>6,779 km</td>      <td>12,742 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg</td>      <td>5.97 × 10^24 kg</td>    </tr>    <tr>      <th>Moons:</th>      <td>2</td>      <td>1</td>    </tr>    <tr>      <th>Distance from Sun:</th>      <td>227,943,824 km</td>      <td>149,598,262 km</td>    </tr>    <tr>      <th>Length of Year:</th>      <td>687 Earth days</td>      <td>365.24 days</td>    </tr>    <tr>      <th>Temperature:</th>      <td>-87 to -5 °C</td>      <td>-88 to 58°C</td>    </tr>  </tbody></table>'

## Mars Hemispheres

In [20]:
hemis_url = "https://marshemispheres.com/"
browser.visit(hemis_url)

In [21]:
html = browser.html
soup = bs(html, 'html.parser')

In [22]:
images = soup.find_all('div', class_="description")

In [23]:
hemisphere_image_urls = []
for image in images:
        # Error handling
        try:
            # Extract href
            href = image.a['href']
            text = image.h3.text
            browser.visit(hemis_url + href)
            
            html=browser.html
            soup=bs(html,'html.parser')
            image_src=soup.find('li').a['href']
        
            image_url = soup.find('img',class_="wide-image")['src']
            print(text + ": \n"+ image_url + "\n")
            
            hemisphere_dict={
                "titles" : text,
                "img_url" :  image_url
            }
            
            hemisphere_image_urls.append(hemisphere_dict)
            
        except Exception as e:
            print(e)


Cerberus Hemisphere Enhanced: 
images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg

Schiaparelli Hemisphere Enhanced: 
images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg

Syrtis Major Hemisphere Enhanced: 
images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg

Valles Marineris Hemisphere Enhanced: 
images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg



In [24]:
browser.quit

<bound method BaseWebDriver.quit of <splinter.driver.webdriver.chrome.WebDriver object at 0x7fef8acbd828>>